#PyQuAcq2

## Install necessary tools

In [ ]:
#!pip install ortools
!pip install exact
#!pip install python-sat
!pip install cpmpy

In [ ]:
import cpmpy as cp

## Define operators

In [ ]:
# Here define constraints
import operator

# this is the relation: a+b!=c
def someAndDiff(a,b,c):
  return operator.ne( operator.add(a,b) , c )

def alldiff():
  pass

def allequ():
  pass

def eqDist1(x,y):

  return operator.eq(operator.abs(operator.sub(x,y)), 1)

def neDist1(x,y):

  return operator.ne(operator.abs(operator.sub(x,y)), 1)



def eqDist(a,b,c,d):
  return operator.eq( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def neDist(a,b,c,d):
  return operator.ne( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))


def eqDist3(repeat,b,c):
  return operator.eq( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

def neDist3(repeat,b,c):
  return operator.ne( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))



vals = [1,2,3,4,5]
ops = [operator.ge,operator.le,operator.lt,operator.gt,operator.ne,operator.eq]


def eqConst(x, const):
  return operator.eq(x,const)

def geConst(x, const):
  return operator.ge(x,const)

def leConst(x, const):
  return operator.le(x,const)

def ltConst(x, const):
  return operator.lt(x,const)

def gtConst(x, const):
  return operator.gt(x,const)

def neConst(x, const):
  return operator.ne(x,const)

## Projection

In [ ]:
def projection(e,vars_ids):
  # this is a projection of Vars on assignement: return the part of the assignement
  # including only Vars

  ids = [ee[0] for ee in e]
  els = { ee[0]: ee for ee in e}
  p = []

  for id in vars_ids:
    if id in ids:
      p.append(els[id])

  return p

## Constraint

In [ ]:
import operator
from copy import deepcopy
class Constraint:
    # this class represents a constraint

    def __init__(self ,scope_ids , rel, arity, isCommutative , parameters = None, isNegation=False, isRedundant=False):
      """
          scope_ids : list of variable ids (e.g. [1,2])
          rel : the relation of this constraint (e.g. operator.ne)
          arity : the arity of the constraint (e.g. 2)
          isCommutative : is this constraint commutative (e.g. True (A != B <=> B != A))
          parameters : list of parameters of this constraint (e.g. A>1 the parameters = [1])
          isNegation : whether this constraint is a negation of the constraint with the same parameters, in other words
          all parameters are the same except the relation which will be replaced by the opposit relation, this will be
          implemented with returning the not of verify later
      """
      assert isinstance(scope_ids,list), "The scope must be a list of variable ids"
      for x in scope_ids:
        assert isinstance(x,int), "The scope must be a list of variable ids"

      self.scope_ids = scope_ids

      self.isCommutative = isCommutative

      #The relation must be an operator
      assert callable(rel), "The relation must be an operator"
      self.rel = rel

      # the arity is the number of variables in the scope, an arity could be binary, ternary ...
      self.arity = arity

      self.parameters = parameters

      self.id = id(self)

      self.isNegation = isNegation
      self.isRedundant = isRedundant

    def verify(self, assignement):
      # this method checks whether the constraint accepts an assignement (True) or rejects it (False)

      # we project the assignement on the scope of the constraint
      p = projection(assignement, self.scope_ids)

      # get values of vars in the assignement
      #print("P: ", p)
      if(p==[]):
        return True # the constraint is not concerned with this assignement

      values = [pp[2] for pp in p]
      # print("values hereeee _________",values)
      # print("Self", self)
      # print("self.rel", self.rel)
      # print("self.scope_ids", self.scope_ids)

      if(len(values) != self.arity and self.arity != None):
        # if the number of the values is different from the arity then just return True because
        # the constraint is not concerned, in general it is unlikely to get here because we do the projection
        # but to avoid errors we keep it here
        return True

      if(self.parameters == None):
        if(len(values)>1):
          if(self.isNegation):
            return not self.rel(*values) # we check if the relation accepts the values given to the variables, if so return not the result because it is a negation
          else:
            return self.rel(*values) # we check if the relation accepts the values given to the variables
        else:
          if(self.isNegation):
            return not self.rel(values[0])
          else:
            return self.rel(values[0])
      else:
        if(len(values)>1):
          if(self.isNegation):
            return not self.rel(*values, *self.parameters)
          else:
            return self.rel(*values, *self.parameters)

        else:
          if(len(self.parameters)>1):
            if(self.isNegation):
              return not self.rel(values[0], *self.parameters)
            else:
              return self.rel(values[0], *self.parameters)
          else:
            if(self.isNegation):
              return not self.rel(values[0], self.parameters[0])
            else:
              return self.rel(values[0], self.parameters[0])

    def prepareCpModelConstraint(self, vars, model):
      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          model += ~ cp.AllDifferent(tmp)
        else:
          model += cp.AllDifferent(tmp)

      elif(self.rel == allequ):
        if(self.isNegation):
          model += ~ cp.AllEqual(tmp)
        else:
          model += cp.AllEqual(tmp)

      elif(self.rel == eqDist1):
        if(self.isNegation):
          model += ~ (abs(tmp[0] - tmp[1]) == 1)
        else:
          model += abs(tmp[0] - tmp[1]) == 1

      elif(self.rel == neDist1):
        if(self.isNegation):
          model += ~(abs(tmp[0] - tmp[1]) != 1)
        else:
          model += abs(tmp[0] - tmp[1]) != 1

      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          model += ~(self.rel(*tmp, *self.parameters))
        else:
          model += self.rel(*tmp, *self.parameters)

      else:
        if(self.parameters!=None):
          if(self.isNegation):
            model += ~(self.rel(*tmp, *self.parameters))
          else:
            model += self.rel(*tmp, *self.parameters)
        else:

          if(self.isNegation):
            model += ~(self.rel(*tmp))  # add the constraint to the model specifying the variables
          else:
            model += self.rel(*tmp)

    def returnCpModelConstraint(self, vars):
      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          return ~ cp.AllDifferent(tmp)
        else:
          return cp.AllDifferent(tmp)

      elif(self.rel == allequ):
        if(self.isNegation):
          return ~ cp.AllEqual(tmp)
        else:
          return cp.AllEqual(tmp)

      elif(self.rel == eqDist1):
        if(self.isNegation):
          return ~ (abs(tmp[0] - tmp[1]) == 1)
        else:
          return abs(tmp[0] - tmp[1]) == 1

      elif(self.rel == neDist1):
        if(self.isNegation):
          return ~(abs(tmp[0] - tmp[1]) != 1)
        else:
          return abs(tmp[0] - tmp[1]) != 1

      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          return ~(self.rel(*tmp, *self.parameters))
        else:
          return self.rel(*tmp, *self.parameters)

      else:
        if(self.parameters!=None):
          if(self.isNegation):
            return ~(self.rel(*tmp, *self.parameters))
          else:
            return self.rel(*tmp, *self.parameters)
        else:

          if(self.isNegation):
            return ~(self.rel(*tmp))  # add the constraint to the model specifying the variables
          else:
            return self.rel(*tmp)

    # def prepareCpModelConstraintReification(self, vars_dict, model, booll):

    #   tmp = []
    #   for  i in self.scope_ids:
    #     tmp.append(vars_dict[i])

    #   if(self.rel == alldiff): #the alldifferent constraint
    #     if(self.isNegation):
    #       model += (~ cp.AllDifferent(tmp)).implies(booll)
    #       model += booll.implies((~ cp.AllDifferent(tmp)))


    #     else:
    #       model += (cp.AllDifferent(tmp)).implies(booll)
    #       model += booll.implies(cp.AllDifferent(tmp))



    #   elif(self.rel == allequ):
    #     if(self.isNegation):
    #       model +=  (~cp.AllEqual(tmp)).implies(booll)
    #       model +=  booll.implies(~cp.AllEqual(tmp))


    #     else:
    #       model += cp.AllEqual(tmp).implies(booll)
    #       model += booll.implies(cp.AllEqual(tmp))


    #   elif(self.rel == eqDist1):
    #     if(self.isNegation):
    #       model +=  (~(abs(tmp[0] - tmp[1]) == 1)).implies(booll)
    #       model +=  booll.implies(~(abs(tmp[0] - tmp[1]) == 1))


    #     else:
    #       model+= (abs(tmp[0] - tmp[1]) == 1).implies(booll)
    #       model +=booll.implies(abs(tmp[0] - tmp[1]) == 1)



    #   elif(self.rel == neDist1):
    #     if(self.isNegation):
    #       model += (~(abs(tmp[0] - tmp[1]) != 1)).implies(booll)
    #       model += booll.implies(~(abs(tmp[0] - tmp[1]) != 1))


    #     else:
    #       model += (abs(tmp[0] - tmp[1]) != 1).implies(booll)
    #       model += booll.implies(abs(tmp[0] - tmp[1]) != 1)



    #   elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
    #     if(self.isNegation):
    #       model += (~(self.rel(*tmp, *self.parameters))).implies(booll)
    #       model += booll.implies(~(self.rel(*tmp, *self.parameters)))


    #     else:
    #       model += (self.rel(*tmp, *self.parameters)).implies(booll)
    #       model += booll.implies(self.rel(*tmp, *self.parameters))



    #   else:
    #     if(self.parameters!=None):
    #       if(self.isNegation):
    #         model += (~(self.rel(*tmp, *self.parameters))).implies(booll)
    #         model += booll.implies(~(self.rel(*tmp, *self.parameters)))


    #       else:
    #         model += (self.rel(*tmp, *self.parameters)).implies(booll)
    #         model += booll.implies(self.rel(*tmp, *self.parameters))


    #     else:
    #       if(self.isNegation):
    #         model += (~(self.rel(*tmp))).implies(booll)
    #         model += booll.implies(~(self.rel(*tmp)))  # add the constraint to the model specifying the variables


    #       else:
    #         model += (self.rel(*tmp)).implies(booll)
    #         model += booll.implies(self.rel(*tmp))


    def prepareCpModelConstraintReification(self, vars_dict, model, booll):

      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars_dict[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          model += cp.all([(~ cp.AllDifferent(tmp)).implies(booll), booll.implies((~ cp.AllDifferent(tmp)))])


        else:
          model+= cp.all([(cp.AllDifferent(tmp)).implies(booll),booll.implies(cp.AllDifferent(tmp))])



      elif(self.rel == allequ):
        if(self.isNegation):
          model += cp.all([(~cp.AllEqual(tmp)).implies(booll),booll.implies(~cp.AllEqual(tmp))])


        else:
          model+= cp.all([cp.AllEqual(tmp).implies(booll),booll.implies(cp.AllEqual(tmp))])


      elif(self.rel == eqDist1):
        if(self.isNegation):
          model+= cp.all([(~(abs(tmp[0] - tmp[1]) == 1)).implies(booll),booll.implies(~(abs(tmp[0] - tmp[1]) == 1))])


        else:
          model+= cp.all([(abs(tmp[0] - tmp[1]) == 1).implies(booll),booll.implies(abs(tmp[0] - tmp[1]) == 1)])



      elif(self.rel == neDist1):
        if(self.isNegation):
          model += cp.all([(~(abs(tmp[0] - tmp[1]) != 1)).implies(booll),booll.implies(~(abs(tmp[0] - tmp[1]) != 1))])

        else:
          model += cp.all([(abs(tmp[0] - tmp[1]) != 1).implies(booll),booll.implies(abs(tmp[0] - tmp[1]) != 1)])



      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          model+= cp.all([(~(self.rel(*tmp, *self.parameters))).implies(booll),booll.implies(~(self.rel(*tmp, *self.parameters)))])


        else:
          model += cp.all([(self.rel(*tmp, *self.parameters)).implies(booll),booll.implies(self.rel(*tmp, *self.parameters))])



      else:
        if(self.parameters!=None):
          if(self.isNegation):
            model += cp.all([(~(self.rel(*tmp, *self.parameters))).implies(booll),booll.implies(~(self.rel(*tmp, *self.parameters)))])

          else:
            model += cp.all([(self.rel(*tmp, *self.parameters)).implies(booll),booll.implies(self.rel(*tmp, *self.parameters))])


        else:
          if(self.isNegation):
            model += cp.all([(~(self.rel(*tmp))).implies(booll),booll.implies(~(self.rel(*tmp)))])

          else:
            model += cp.all([(self.rel(*tmp)).implies(booll),booll.implies(self.rel(*tmp))])






    def isEqualTo(self, constraint):

      # vars_ids, vars_domains, vars_types Are globals

      # if(not isinstance(constraint, Constraint)):
      #   return False

      if(isinstance(constraint, Conjunction)):
        conjScp = set(constraint.scope)
        if(set(self.scope_ids) != conjScp):
          return False

        v = set(self.scope_ids).union(conjScp)

        # #print("\n The union of the scope: ", v)
        prob_data = {}
        for vv in v:
          prob_data[vv] = problem_data[vv]
          #prob_data[vv]["domain"] = (0,len(v))

        # # print("\n The prob_data: ", prob_data.keys())

        # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

        n1 = Network(
                 prob_data,
                 [self])

        n2 = Network(
                      prob_data,
                      [constraint])

        e = n1.solve()

        if(not n2.isAccepted(e)):
          return False

        e = n2.solve()

        if(not n1.isAccepted(e)):
          return False

        notself = deepcopy(self)
        notself.isNegation = True

        notconstraint = deepcopy(constraint)
        notconstraint.isNegation = True

        if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
          return True

        return False


        # # return listOfSolsEqual(n1.getAllSolutions(), n2.getAllSolutions())

        # # elementaryConsts = deepcopy(constraint.elementaryConstraints)
        # notc = deepcopy(self)
        # notc.isNegation = True

        # # # print("\nElementary constraints of the conjunction: \n")
        # # # for c in elementaryConsts:
        # # #   print(c.rel, c.scope_ids)

        # # elementaryConsts.append(notc)
        # # n = Network(prob_data, elementaryConsts)
        # n = Network(prob_data, [constraint,notc])


        # # # print("Notc: \n")
        # # # print(notc.rel, notc.scope_ids, notc.isNegation)

        # # # print(n.solve())
        # # # print(input())
        # if(n.solve()==[]):
        #   return True
        # else:
        #   return False

      if self.id == constraint.id:
        return True
      else:

        conjScp = set(constraint.scope_ids)
        if(set(self.scope_ids) != conjScp):
          return False

        v = set(self.scope_ids).union(conjScp)

        # #print("\n The union of the scope: ", v)
        prob_data = {}
        for vv in v:
          prob_data[vv] = problem_data[vv]
          #prob_data[vv]["domain"] = (0,len(v))

        # # print("\n The prob_data: ", prob_data.keys())

        # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

        n1 = Network(
                  prob_data,
                  [self])

        n2 = Network(
                      prob_data,
                      [constraint])

        e = n1.solve()

        if(not n2.isAccepted(e)):
          return False

        e = n2.solve()

        if(not n1.isAccepted(e)):
          return False

        notself = deepcopy(self)
        notself.isNegation = True

        notconstraint = deepcopy(constraint)
        notconstraint.isNegation = True

        if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
          return True

        return False


def listOfSolsEqual(l1,l2):

  if(l1 == [] and l2 == []):
    return True

  if(l1 != [] and l2 == []):
    return False

  if(l1 == [] and l2 != []):
    return False

  if len(l1) != len(l2):
    return False


  cpt1=0
  for a in l1:
    if a in l2:
      cpt1+=1

  cpt2=0
  for a in l2:
    if a in l1:
      cpt2+=1
  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False

## Conjunction

In [ ]:
import numpy as np


class Conjunction():

  def __init__(self, constraints, isNegation =False, isRedundant=False):
    # a conjunction is the AND of constraints

    self.scope_ids = [c.scope_ids for c in constraints]

    self.rel = [c.rel for c in constraints]

    self.constraints = constraints

    self.elementaryConstraints = self.AllElementaryConstraints()

    self.scope = self.getScope()

    #self.id = sum([c.id for c in self.constraints])
    self.id = id(self)

    self.isNegation = isNegation
    self.isRedundant= isRedundant

  def getScope(self):

    scopes = []

    for c in self.constraints:
      if isinstance(c,Constraint):
        scopes.append(set(c.scope_ids))
      else:
        scopes.append(set(c.scope))

    ss = list(set.union(*scopes))

    return list(np.unique(ss))


  def AllElementaryConstraints(self):

    elementaryConstraints = []

    for c in self.constraints:
      if isinstance(c,Constraint):
        elementaryConstraints.append(c)
      else:
        elementaryConstraints+= c.elementaryConstraints

    return elementaryConstraints


  def isEqualTo(self, conj):

    # if(not isinstance(conj, Conjunction)):
    #   return False

    if(isinstance(conj, Constraint)):
      return conj.isEqualTo(self)

    if self.id == conj.id:
      return True
    else:
      conjScp = set(conj.scope)
      if(set(self.scope) != conjScp):
        return False

      v = set(self.scope).union(conjScp)

      # #print("\n The union of the scope: ", v)
      prob_data = {}
      for vv in v:
        prob_data[vv] = problem_data[vv]
        #prob_data[vv]["domain"] = (0,len(v))

      # # print("\n The prob_data: ", prob_data.keys())

      # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

      n1 = Network(
                prob_data,
                [self])

      n2 = Network(
                    prob_data,
                    [conj])

      e = n1.solve()

      if(not n2.isAccepted(e)):
        return False

      e = n2.solve()

      if(not n1.isAccepted(e)):
        return False

      notself = deepcopy(self)
      notself.isNegation = True

      notconstraint = deepcopy(conj)
      notconstraint.isNegation = True

      if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
        return True

      return False



  def verify(self, assignement,liste = []):
    # for a conjunction to accept an assignement, all constraints from which
    # it is built must accept it (i.e there is an AND between these constraints )

    liste1 = []
    for c in self.constraints:
      if(isinstance(c,Constraint)):
        liste1.append(c.verify(assignement))

    liste2 = []
    for c in self.constraints:
      if(not isinstance(c,Constraint)):
        liste2.append(c.verify(assignement,liste2))

    listtt = liste1 + liste2
    if (listtt == []):
      return all([listtt]) # if only one constraint doesn't accept the assignement this returns False. It returns True otherwise
    else:
      res = all(listtt)
      listtt = []
      return res

  def prepareCpModelConstraint(self, vars_dict, model):

    # This method adds the constraints in this conjunction to the model
    if not self.isNegation:
      ec = deepcopy(self.elementaryConstraints)
      for c in ec:
        model+= cp.all([c.returnCpModelConstraint(vars_dict) for c in ec])
    else:
      ec = deepcopy(self.elementaryConstraints)
      for c in ec:
        c.isNegation = True
      model+= cp.any([c.returnCpModelConstraint(vars_dict) for c in ec])

  # def prepareCpModelConstraintReification(self,vars_dict,model,booll):
  #   model+= (cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])).implies(booll)
  #   model+= booll.implies(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints]))

  def prepareCpModelConstraintReification(self,vars_dict,model,booll):
    model += cp.all([(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])).implies(booll),booll.implies(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])) ])



  def isScopeIncludedInY(self,Y, liste = []):
  # this method verifies if the scope is included in Y
  # the scope of a conjunction is all the variables in the scopes of the constraints from which it is built

    scope = self.scope

    if set(scope).issubset(set(Y)):
      return True

    return False


  def isScopeIsExactlyY(self,Y):

    # this method verifies if the scope is Exactly Y
    scope = self.scope
    return set(scope) == set(Y)


In [ ]:
# problem_data = {
#     1:{
#       "domain":(1,5),
#        "type":"d",
#        "name":"A"
#     },
#     2:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"B"
#     },
#     3:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"C"
#     },
#     4:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"D"
#     },
#     5:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"E"
#     },
# }


# # c = Conjunction([
# #     Constraint([1,2], operator.eq, 2, True),
# #     Constraint([1,3], operator.eq, 2, True),
# #     Conjunction([
# #       Constraint([1,4], operator.eq, 2, True),
# #       Constraint([1,5], operator.eq, 2, True),
# #     ])
# # ])

# # n1 = Network(problem_data, [c])
# # print("before: ", n1.solve())

# # c.isNegation = True
# # n2 = Network(problem_data, [c])
# # print("after: ", n2.solve())



# c = Conjunction([
#     Constraint([1,3], operator.ne, 2, True),
#     Constraint([1,3], operator.ge, 2, False),
# ])



# co = Constraint([1,3], operator.gt, 2, False)


# co.isEqualTo(c)

## Network

In [ ]:
from ortools.sat.python import cp_model
from itertools import permutations, combinations

# This was borrowed from ortools official documentation, it catches all the solutions
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, Limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solutions = []
        self.Limit = Limit

    def on_solution_callback(self):
        self.__solution_count += 1

        e = []
        self.__solution_count+=1
        for v in self.__variables.keys():
          e.append(
              (v, self.__variables[v].Name() , self.Value(self.__variables[v]))
          )
        #print(f"Solution {self.__solution_count}: ",e)

        self.__solutions.append(e)

        if(self.Limit != None):
          if(self.__solution_count > self.Limit):
             self.StopSearch()

    def solution_count(self):
        return self.__solution_count

    def allSolutions(self):
      return self.__solutions

In [ ]:
class Network():

  def __init__(self, problem_data ,constraints):

    self.problem_data = problem_data

    self.constraints = constraints

    self.all_solutions = []

  def isAccepted(self, e):
    # An assignement e is accepted by a network if it doesn't violate any constraints of the network

    return all( [c.verify(e) for c in self.constraints])

  def isSolution(self,e):
    # e is a solution if it is complete and accepted
    ids = [ee[0] for ee in e]
    return self.isAccepted(e) and set(ids) == set(self.problem_data.keys())


  def ConstraintsIncludedInY(self,Y):
    # This method constructs a network from the constraints
    # that have scope included in Y

    res = []

    for c in self.constraints:
      if(isinstance(c,Constraint)):
        if set(c.scope_ids).issubset(set(Y)):
            res.append(c)
      else:
        if(c.isScopeIncludedInY(Y)):
          res.append(c)

    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]


    return Network(prob_data,res)


  def ConstraintsIsExactlyY(self,Y):
    # This method constructs a network from the constraints
    # that have scope is Exactly Y

    res = []

    for c in self.constraints:
      if(isinstance(c,Constraint)):
        if set(c.scope_ids) == set(Y) :
          res.append(c)
      else:
        if(c.isScopeIsExactlyY(Y)):
          res.append(c)


    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]


    return Network(prob_data,res)


  def solve(self):

    model1 = cp.Model()

    vars = {id: cp.intvar(self.problem_data[id]["domain"][0],self.problem_data[id]["domain"][1], shape=1, name=self.problem_data[id]["name"] )  for id in self.problem_data.keys()}

    for c in self.constraints:
      c.prepareCpModelConstraint(vars,model1)

    for v in vars.keys():
        model1 += vars[v] >= vars[v].lb  # this is necessary because cpmpy doesn't instantiate vars that are not used in any constraint


    if(model1.solve(solver="exact")):
      e = []
      for v in vars.keys():
        e.append(
            (v, vars[v].name, vars[v].value())
          )
      return e
    else:
        #print('No solution found.')
        return []


  def allSolutions(self, TimeLimit = None, SolutionLimit= None):
    self.all_solutions = []

    model2 = cp.Model()

    vars = {id: cp.intvar(self.problem_data[id]["domain"][0],self.problem_data[id]["domain"][1], shape=1, name=self.problem_data[id]["name"] )  for id in self.problem_data.keys()}


    for c in self.constraints:
      c.prepareCpModelConstraint(vars,model2)

    for v in vars.keys():
        model2 += vars[v] >= vars[v].lb

    model2.solveAll(solver="exact",display = lambda : self.all_solutions.append([(k,vars[k].name, vars[k].value()) for k in vars.keys()]), time_limit= TimeLimit, solution_limit = SolutionLimit)

  def getAllSolutions(self, TimeLimit = None, SolutionLimit= None):
    # this is the function which will be used in other places.
    self.allSolutions(TimeLimit, SolutionLimit)

    return self.all_solutions

  # def isEquivalentTo(self, N):
  #   # If the sets of solutions of two networks T and T2 are equal, then these two networks are equivalent
  #   e1 = self.solve()
  #   if(not N.isAccepted(e1)):
  #     return False

  #   e2 = N.solve()
  #   if(not self.isAccepted(e2)):
  #     return False

  #   selfsols = self.getAllSolutions()
  #   for solution in selfsols:
  #     if not N.isAccepted(solution):
  #       return False

  #   return True



  def networkOfConstraintsThatRejectE(self,e):
    # Here we construct a network containing constraints that reject e
    tmp = []

    Y = set()
    for c in self.constraints:
      if not c.verify(e):
        tmp.append(c)

        if(isinstance(c,Conjunction)):
          Y = Y.union(set(c.scope))
        else:
          Y = Y.union(set(c.scope_ids))




    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]



    return Network(prob_data, tmp)


  def removeConstraintsThatRejectE(self,e):
    # we remove the constraints that reject e from the current network
    tmp = []

    for c in self.constraints:
      if c.verify(e):
        tmp.append(c)

    self.constraints = tmp
    return self


  def addConstraint(self,c):
    # we add  the constraint c to the network
    self.constraints.append(c)
    return self

  def removeListOfConstraints(self, l):
    # we remove a list l of constraints from the current network

    tmp = []
    for c in self.constraints:
      #if c not in l:
      if not constraintinConstraints(c,l):
        tmp.append(c)

    self.constraints = tmp

    return self

  def removeRedundants(self):
    tmp = []
    for c in self.constraints:
      #if c not in l:
      if not c.isRedundant:
        tmp.append(c)

    self.constraints = tmp

    return self


  def isEqualTo(self,constraint1, constraint):

    if(not isinstance(constraint, Constraint)):
        return False

    if(constraint1.arity == constraint.arity and  constraint1.parameters == constraint.parameters and constraint1.rel == constraint.rel):

      if(constraint1.scope_ids == constraint.scope_ids):
        return True

      else:
        if(set(constraint1.scope_ids) == set(constraint.scope_ids)):

          if(constraint1.isCommutative and constraint.isCommutative):
            return True


          v = set(constraint1.scope_ids).union(set(constraint.scope_ids))


          prob_data = {}
          for vv in v:
            prob_data[vv] = problem_data[vv]


          n1 = Network(
                    prob_data,
                    [constraint1])

          n2 = Network(
                    prob_data,
                    [constraint])

          return listOfSolsEqual(n1.getAllSolutions(), n2.getAllSolutions())

    return False



  def removeDuplicates(self):
    res = []
    tmp = []

    for c in self.constraints:
      if not self.existAlready2(res,c):
        tmp.append(c)

    self.constraints = tmp
    return self


  def existAlready2(self,res,c):
    if(self.ConstraintinListOfConstraints(c, res)):
      return True
    res.append(c)
    return False

  def ConstraintinListOfConstraints(self,a,listOfConsts):
    for l in listOfConsts:
      if(a.isEqualTo(l)):
        return True

    return False

  def isEquivalentTo(self, net):

    a1 = []
    for c in self.constraints:
      a1.append(isImplied(c, net.constraints))

    a2 = []
    for c in net.constraints:
      a2.append(isImplied(c,self.constraints))

    return all(a1) and all(a2)



def listOfSolsEqual(l1,l2):

  if(l1 == [] and l2 == []):
    return True

  if(l1 != [] and l2 == []):
    return False

  if(l1 == [] and l2 != []):
    return False

  if len(l1) != len(l2):
    return False


  cpt1=0
  for a in l1:
    if a in l2:
      cpt1+=1

  cpt2=0
  for a in l2:
    if a in l1:
      cpt2+=1
  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False

def constraintinConstraints(c,l):

  for cc in l:
    if c.isEqualTo(cc):
      return True

  return False

def isImplied(c,consts):
  n = []
  n = Network(problem_data, deepcopy(consts))
  notc = deepcopy(c)
  notc.isNegation = True

  n= n.addConstraint(notc)

  if(n.solve()==[]):
    return True

  return False


## Basis

In [ ]:
from itertools import permutations, combinations

#The language is like this = [(rel,arity, isCommutative, parameters)]


class Basis(Network):
  # A Basis is also a network, but its constraints are built from a language

  def __init__(self,problem_data, language):
    super().__init__(problem_data,[])
    self.language = language

  def build(self):

   # we build the set of constraints using permutations of variables of length == arity of relation (operator)
    if((operator.lt,2,False,None) in self.language and (operator.gt,2,False,None) in self.language):
      self.language.remove((operator.gt,2,False,None))

    if((operator.le,2,False,None) in self.language and (operator.ge,2,False,None) in self.language):
      self.language.remove((operator.ge,2,False,None))

    for l in self.language:
      if(l[1] != None): # this is a bounded constraint
        if(l[1] == 1): # the unary constraints with a parameter
          # if(l[0] in [eqConst, neConst,ltConst, leConst, gtConst, geConst]):
          #   for v in self.vars_ids:
          #     for param in [1,2,3,4,5]:
          #       self.constraints.append(Constraint([v],l[0],l[1],l[2],[param]))

          if(l[3] != None):
            for v in self.problem_data.keys():
              for param in l[3]:
                self.constraints.append(Constraint([v],l[0],l[1],l[2],[param]))
        else:
          if(l[2] == False): # Not commutative, then we need both [a,b] and [b,a] so we do permutations
            for p in permutations(self.problem_data.keys(),l[1]):
              self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
          else: #Commutative, just [a,b] is enough so we use combinations
            if(l[0] in [eqDist, eqDist3, neDist, neDist3]):
              for p in combinations(self.problem_data.keys(),l[1]):
                aa = np.array(list(p))
                if(len(np.unique(aa)) == len(aa)):
                  self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
            else:
              for p in combinations(self.problem_data.keys(),l[1]):
                self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))


      else: # this is an unbounded constraint(global)
        ll = 2
        while( ll<=len(self.problem_data.keys())):
          for p in permutations(self.problem_data.keys(),ll):
            self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
          ll+=1

## Send a query

In [ ]:
def ask(e, Target):
  global Qc
  global Qa
  global oldsize
  global newsize
  global maxwaittimeold
  global waittimes
  global Examplesizes

  now = time.time()

  waittimes.append(now-maxwaittimeold)
  maxwaittimeold = now

  Qc+=1
  Examplesizes.append(len(e))

  if(newsize > oldsize):
    oldsize = newsize
    if(Qc > Qa):
      Qa = Qc



  if set([ee[0] for ee in e]) == set(Target.problem_data.keys()):
    # This is a membership query
    #print("Hereeeee 367 : ", e)
    #if e in Target.getAllSolutions(): # not necessary and when we generate just for example 10000 solutions, this will return false if
                                       # e is not in the first 10000 even if it is a solution
    if Target.isAccepted(e): # we don't need all solutions here !!!!!!!!!, just verify if it is accepted or not !
      print("\n a COMPLETE query was sent, the answer was YES")
      #print("The example was : \n\n", e)
      return True
    else:
      print("\n a COMPLETE query was sent, the answer was NO")
      # print("The Negative Example was : \n\n", e)
      # print(input())
      return False

  else:
    # This is a partial query
    #print("Hereeeee 376 : ", e)
    if Target.isAccepted(e):
      print("\n a PARTIAL query was sent, the answer was YES")
      #print("The example was : \n\n", e)
      return True
    else:
      print("\n a PARTIAL query was sent, the answer was NO")
      # print("The Negative Example was : \n\n", e)
      # print(input())
      return False

## Generate Example with cuttoff

In [ ]:
import time


def solve(n, scope):

  st = time.time()
  b = n.ConstraintsIncludedInY(scope)

  r = b.solve()
  if( r == []):
    return [], time.time()-st
  else:
    return projection(r,scope) , time.time()-st
    #return r , time.time()-st

def getVarIdBdeg(X):
  global B
  global vars_ids
  nbConsts = 0
  idd = None
  for x in X:
    cpt = 0
    for c in B.constraints:
      if(isinstance(c, Conjunction)):
        if(x in c.scope):
          cpt+=1
      else:
        if(x in c.scope_ids):
          cpt+=1

    if(cpt>nbConsts):
      # print("\n New max consts: ", cpt)
      # print("\n Var id ", x)
      nbConsts = cpt
      idd = x

  # print("\n Final max consts: ", nbConsts)
  # print("\n Var id ", idd)
  return idd




def solveCuttOffCore(n, scope, remainingTime):
  Y = scope[:]
  best = None
  X = list(set(problem_data.keys()).difference(set(Y)))[:]

  start = time.time()


  while(X != []):



    #Y.append(X[0]) # use bdeg heuristic
    i = getVarIdBdeg(X)
    if(i!=None):
      Y.append(i)
    else:
      Y.append(X[0])
    #print("\n\n Next Y: ", Y)
    X = list(set(problem_data.keys()).difference(set(Y)))
    #print("\n\n Next X: ", X)
    Y = sorted(Y)
    b = n.ConstraintsIncludedInY(Y)

    model3 = cp.Model()

    vars = {id: cp.intvar(b.problem_data[id]["domain"][0],b.problem_data[id]["domain"][1], shape=1, name=b.problem_data[id]["name"] )  for id in b.problem_data.keys()}

    for c in b.constraints:
      c.prepareCpModelConstraint(vars,model3)

    for v in vars.keys():
        model3 += vars[v] >= vars[v].lb  # this is necessary because cpmpy doesn't instantiate vars that are not used in any constraint

    if(model3.solve(solver="exact",time_limit=remainingTime) ):
      s = []
      for v in vars.keys():
        s.append(
            (v, vars[v].name, vars[v].value())
          )
    else:
      if(model3.cpm_status.exitstatus.name == "UNSATISFIABLE"):
        return []
      else:
        return None


      # if time.time() - start > remainingTime:
      #   return best

    for v in X:
      #print("\n\n\n Inside if leads to arc inconsistency, remainingtime was : ",remainingTime)
      #print("\n Inside if leads to arc inconsistency, Time consumed so far : ", time.time() - start)
      #print("\nExit condition : ", time.time() - start > remainingTime)
      # if time.time() - start > remainingTime:
      #   #print("HERE 46 will return None")
      #   return None

      tmp = n.ConstraintsIsExactlyY(Y+[v])
      tmp2 = n.ConstraintsIsExactlyY([v]+Y)

      if(tmp.solve()==[] or tmp2.solve==[]):
        #leads to arc inconsistency
        return []

      # #print("\nExit condition : ", time.time() - start > remainingTime)
      # if time.time() - start > remainingTime:
      #   return None
  #best = projection(s,Y)
    best = s

    # print("\n\n\nRemainingtime was : ",remainingTime)
    # print("\nTime consumed so far : ", time.time() - start)

    # print("\nExit condition : ", time.time() - start > remainingTime)
    if time.time() - start > remainingTime:
      #print("HERE 58 will return None")
      return best
  return best

def solveCuttOff(n,scope, remainingTime):


  started = time.time()
  s = solveCuttOffCore(n,scope, remainingTime)

  #print("\nAfter solveCuttOffCore:\n return value : ", s, " remainingTime: ", remainingTime)
  #print("It took: ", time.time() - started)

  if(s == None):
    return (None, remainingTime)
  else:
    return (s, time.time() - started)




In [ ]:
import time
from copy import deepcopy

redundants = []
def generateExample(X,L, B):
  global redundants
  cuttoff = 1
  timer = 0



  cpt = 0
  while(cpt<len(B.constraints)):
    cc = deepcopy(B.constraints[cpt])
    cc.isNegation = True
    ll = deepcopy(L)
    ll = ll.addConstraint(cc)

    e,t = solve(ll, cc.scope_ids)

    timer+=t

    if e==[]:
      redd = B.constraints[cpt]
      redd.isRedundant = True
      redundants.append(redd)
      L = L.addConstraint(redd)
      B = B.removeListOfConstraints([redd])
    else:
      rt = cuttoff - timer
      if(rt<0):
        rt = 0
      ep, tp = solveCuttOff(ll, cc.scope_ids, rt)
      timer += tp

      if ep == None:
        return e

      if ep == []:
        redd = B.constraints[cpt]
        redd.isRedundant = True
        redundants.append(redd)
        L = L.addConstraint(redd)
        B = B.removeListOfConstraints([redd])
      else:
        return ep
  print("Redundants: \n")
  for r in redundants:
    print(r)

  print("before remove reudndants: \n")
  print(len(L.constraints))
  #L = L.removeListOfConstraints(redundants)
  L = L.removeRedundants()
  print("after remove reudndants: \n")
  print(len(L.constraints))
  return []


## FindScope

In [ ]:
def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False
def areListsOfConstraintsEqual(l1,l2):
  if(l1 == [] and l2!=[]):
    return False
  if(l1 != [] and l2==[]):
    return False
  if(l1 == [] and l2==[]):
    return True
  if(len(l1)!=len(l2)):
    return False

  cpt1  = 0
  for l in l1:
    if ConstraintinListOfConstraints(l,l2):
      cpt1+=1

  cpt2  = 0
  for l in l2:
    if ConstraintinListOfConstraints(l,l1):
      cpt2+=1

  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False


def FindScope(e, R ,Y , B):
  global calls
  global Target

  #print("\n\nFind scope call ", calls)
  # calls+=1

  # print("R: ", [r for r in R])
  # print("Y: ", [y for y in Y])

  # if(calls > 20):
  #   print("Possible Recusion Error ") # For test purpose, to be removed later
  #   return

  if B.networkOfConstraintsThatRejectE(projection(e,R)).constraints != [] :
    if(ask(projection(e,R), Target)):
      # print("Here")
      # print("In FindScope B[",Y,"] before removal: \n")
      # for c in B.ConstraintsIncludedInY(Y).constraints:
      #   print(c.scope_ids, c.rel)

      B = B.removeConstraintsThatRejectE(projection(e,R))
      print("\nInside FindScope B reduced in size, new size: ",len(B.constraints))
      # print("In FindScope B[",Y,"] After removal: \n")
      # for c in B.ConstraintsIncludedInY(Y).constraints:
      #   print(c.scope_ids, c.rel)
    else:
      # print("Here 2")
      return []

  if len(Y) == 1:
    return Y

  l = len(Y)

  l2 = l//2

  if l == 2 :
    Y1 = Y[:l2]
    Y2 = Y[l2:]
  else:
    Y1 = Y[:l2+1]
    Y2 = Y[l2+1:]


  if(areListsOfConstraintsEqual(B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y1))))).constraints , B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y))))).constraints ) ):
    #print("Here !!! 1")
    S1 = []
  else:
    #print("Here !!! 2")
    S1 = FindScope(e, list(set(R).union(set(Y1))), Y2, B )

  if( areListsOfConstraintsEqual(B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(S1))))).constraints , B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y))))).constraints) ):
    #print("Here !!! 3")
    S2 = []
  else:
    #print("Here !!! 4")
    S2 = FindScope(e, list(set(R).union(set(S1))), Y1, B)


  return list( set(S1).union(set(S2)) )


## Join Networks

In [ ]:
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def existAlready(res,nc1,nc2):
    # conj = joinConstraints([],nc1,nc2)
    # conj2 = joinConstraints([],nc2,nc1)
    conj = joinConstraints([],nc1,nc2)
    if(ConstraintinListOfConstraints(conj, res)):
        return True
    # if(ConstraintinListOfConstraints(conj2, res)):
    #     return True

    return False



def existAlready2(res,c):

    if(ConstraintinListOfConstraints(c, res)):
        return True
    res.append(c)
    return False


def inListOfLists(a,listOfLists):

  for l in listOfLists:
    if(isinstance(l,int)):
      if(l == a):
        return True
    else:
       res = inListOfLists(a,l)
       if(res):
         return True

  return False


def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False

import time
def areIncompatible(vars_data,c1,c2):


  scope1 = c1.scope_ids if isinstance(c1,Constraint) else c1.scope

  scope2 = c2.scope_ids if isinstance(c2,Constraint) else c2.scope

  scope = set(scope1).union(set(scope2))

  vd = {}

  for id in vars_data.keys():
    if id in scope:
      vd[id] = vars_data[id]

  n = Network(
      vd,
      [c1,c2])

  # print("\n\nThe network inside areIncompatible: \n")
  # print("VarsData: \n")
  # print(n.problem_data)
  # print("Constraints: \n")

  # for c in n.constraints:
  #   print(c.rel,c.scope_ids)

  # st = time.time()
  if(n.solve() == []):
    # print("To prove incompatibility, it took: ", time.time()-st)
    return True

  # print("To prove incompatibility, it took: ", time.time()-st)
  return False


# def joinConstraints(res,c1,c2):
#   if(c1.isEqualTo(c2)):
#     return c1
#   else:
#     res.append(Conjunction([c1,c2]))
#     return Conjunction([c1,c2])


def joinConstraints(res,c1,c2):
  # you can add incompatibility test here to do just one pass
  if(c1.isEqualTo(c2)):
    return c1
  else:
    if(isinstance(c2,Conjunction) and isinstance(c1,Constraint)):
      if(not ConstraintinListOfConstraints(c1, c2.elementaryConstraints)):
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
      else:
        res.append(c2)
        return c2
    elif(isinstance(c1,Conjunction) and isinstance(c2,Constraint)):
      if(not ConstraintinListOfConstraints(c2, c1.elementaryConstraints)):
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
      else:
        res.append(c1)
        return c1
    elif(isinstance(c1,Conjunction) and isinstance(c2,Conjunction)):
      if(ConstraintinListOfConstraints(c1, c2.constraints)):
        res.append(c2)
        return c2
      elif(ConstraintinListOfConstraints(c2, c1.constraints)):
        res.append(c1)
        return c1
      else:
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
    else:
      res.append(Conjunction([c1,c2]))
      return Conjunction([c1,c2])




def joinNetworks(N1,N2):


  if len(N2.constraints) == 0:
    return N1



  #vars_data = [(i,d,t) for i,d,t in zip(N1.vars_ids,N1.vars_domains,N1.vars_types)]

  vars_data = N1.problem_data

  already_ids = [k for k in vars_data.keys()]

  for id in N2.problem_data.keys():
    if(id not in already_ids):
      vars_data[id] = N2.problem_data[id]

  res = []

  df1 = pd.DataFrame({"consts": N1.constraints})
  df2 = pd.DataFrame({"consts": N2.constraints})



  dfr = df1.join(df2, lsuffix='_df1', rsuffix='_df2', how="cross")

  #print("len(delta):\n", len(dfr))
  #tqdm.pandas()
  res = []



  #dfr["Conjunction"] = dfr.progress_apply(lambda x: joinConstraints(res,x.consts_df1, x.consts_df2) if not existAlready(res,x.consts_df1, x.consts_df2) else "eee", axis=1)
  dfr["Conjunction"] = dfr.apply(lambda x: joinConstraints(res,x.consts_df1, x.consts_df2) if not existAlready(res,x.consts_df1, x.consts_df2) and not areIncompatible(vars_data,x.consts_df1, x.consts_df2) else "eee", axis=1)


  dfr = dfr[dfr["Conjunction"] != "eee"]

  #dfr["AreIncompatible"] = dfr.progress_apply(lambda x: areIncompatible(vars_data,x.consts_df1, x.consts_df2), axis=1)
  # dfr["AreIncompatible"] = dfr.apply(lambda x: areIncompatible(vars_data,x.consts_df1, x.consts_df2), axis=1)


  # dfr = dfr[dfr["AreIncompatible"] == False]

  res = []
  #dfr["Conjunction_unique"] = dfr["Conjunction"].progress_apply(lambda x: x if not existAlready2(res,x) else "eee")
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")

  dfr = dfr[dfr["Conjunction_unique"] != "eee"]

  return Network(vars_data,dfr["Conjunction_unique"].values)


<ipython-input-15-4d80adbd618e>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [ ]:
# def joinNetworks(n1,n2):
#   if(len(n1.constraints)==0):
#     return n2

#   if(len(n2.constraints)==0):
#     return n1


#   vars_data = n1.problem_data

#   already_ids = [k for k in vars_data.keys()]

#   for id in n2.problem_data.keys():
#     if(id not in already_ids):
#       vars_data[id] = n2.problem_data[id]

#   resConjs = []
#   resElementary = []

#   for c1 in n1.constraints:
#     for c2 in n2.constraints:
#       if(not c1.isEqualTo(c2) and Network(vars_data, [c1,c2]).solve()!=[]):
#         conj = Conjunction([c1,c2])
#         exist = False
#         for co in resConjs:
#           if(co.isEqualTo(conj)):
#             #exist already
#             exist = True
#             break

#         if(not exist):
#           resConjs.append(conj)

#       else:
#         existE = False

#         for e in resElementary:
#           if(e.isEqualTo(c1)):
#             existE = True
#             break

#         if(not existE):
#           implied = False
#           for co in resConjs:
#             notc = deepcopy(c1)
#             notc.isNegation = True
#             if( Network(vars_data, [co, notc]).solve==[]):
#               implied=True
#               break

#           if not implied:
#             resElementary.append(c1)



#   return Network(vars_data, resConjs + resElementary)


## FindEPrime

In [ ]:
# from tqdm import tqdm
# def isListConstsSubSetOtherListConsts(a,b):
#   if(a == []):
#     return True

#   if(a == [] and b== []):
#     return True

#   if(a != [] and b == []):
#     return False

#   if(len(a)>len(b)):
#     return False


#   for aa in tqdm(a, ascii="True", desc="SubList"):
#     cpt = 0
#     for bb in b:
#       if(aa.isEqualTo(bb)):
#         cpt+=1

#     if(cpt==0):
#       return False

#   return True


# from random import random



# # This was borrowed from ortools official documentation, it catches all the solutions
# class SolutionsCallBack(cp.solvers.ortools.OrtSolutionCounter):


#     def __init__(self, solver ,variables, delta):
#         cp.solvers.ortools.OrtSolutionCounter.__init__(self)


#         self.__solution_count = 0
#         self.__solutions = []

#         self.delta = delta
#         self.ds = len(delta.constraints)
#         self._cpm_vars = solver.user_vars
#         self._varmap = solver._varmap

#         self.solver_vars = solver.solver_vars(list(self._cpm_vars))

#         self.__variables = variables

#         self.variables2 = {}

#         for k in self.__variables.keys():
#           for v in self.solver_vars:
#             if self.__variables[k].name == v.Name():
#               self.variables2[k] = v


#         self.solver = solver


#         self.example = None



#     def on_solution_callback(self):
#         super().on_solution_callback()
#         self.__solution_count+=1

#         e = []
#         for v in self.variables2.keys():
#           e.append(
#               (v, self.variables2[v].Name() , self.Value(self.variables2[v]))
#           )

#         print("\n Inside Find Eprime Solution nb: ", self.solution_count())

#         d = self.delta.networkOfConstraintsThatRejectE(e).constraints

#         if( d!=[] and len(d)!= self.ds ):
#           self.example = e
#           self.StopSearch()

#         # print("solution nb ",self.solution_count()," : ",e)

#     def solution_count(self):
#         return self.__solution_count

#     def allSolutions(self):
#       return self.__solutions

# # def deltaIsImpliedByLY(delta,ll):


# #   cpt=0
# #   while(cpt<len(delta.constraints)):
# #     notc = deepcopy(delta.constraints[cpt])
# #     notc.isNegation = True

# #     lll = deepcopy(ll)

# #     lll = lll.addConstraint(notc)

# #     e = lll.solve()

# #     if e == []:
# #       cpt+=1
# #     else:
# #       return False

# #   return True


# def findEPrime(L,Y,delta):

#   ds = len(delta.constraints)

#   if(ds==0 or ds==1):
#     return [], None, None


#   ll = L.ConstraintsIncludedInY(Y)

#   # if deltaIsImpliedByLY(delta,ll):
#   #   return []


#   model = cp.Model()

#   #vars = { id:  cp.intvar(dom[0],dom[1], shape=1, name=t+str(id))  for id,dom,t in zip(ll.vars_ids, ll.vars_domains, ll.vars_types) }
#   vars = { id:  cp.intvar(ll.problem_data[id]["domain"][0],ll.problem_data[id]["domain"][1], shape=1, name=ll.problem_data[id]["name"])  for id in ll.problem_data.keys()}
#   for c in ll.constraints:
#     c.prepareCpModelConstraint(vars,model)

#   for v in vars.keys():
#       model += vars[v] >= vars[v].lb

#   s = cp.SolverLookup.get("ortools", model) # faster on a solver interface directly
#   cb = SolutionsCallBack(s, vars, delta)

#   s.solve(time_limit=20,enumerate_all_solutions= True, solution_callback=cb)

#   if(cb.example != None):
#     return cb.example, ll, delta
#   else:
#     return findEPrimeReification(L,Y,delta)


In [ ]:
# def solve2(ll,scope):

#   #b = ll.ConstraintsIncludedInY(scope)
#   r = ll.solve()
#   if( r == []):
#     return []
#   else:
#     #return projection(r,scope)
#     return r

# redundants2 = []
# def findEPrime(L,Y,d):

#   delta = deepcopy(d)
#   ll = deepcopy(L.ConstraintsIncludedInY(Y))


#   while(len(delta.constraints)!=0):
#     notc = deepcopy(delta.constraints[0])

#     notc.isNegation = True

#     ll = ll.addConstraint(notc)
#     res = solve2(ll, notc.scope_ids ) if isinstance(notc, Constraint) else solve2(ll, notc.scope )

#     if(res == []):
#       redundants2.append(delta.constraints[0])
#       ll = ll.addConstraint(delta.constraints[0])
#       delta = delta.removeListOfConstraints([delta.constraints[0]])
#     else:
#       if(len(d.networkOfConstraintsThatRejectE(res).constraints)== 0 or len(d.networkOfConstraintsThatRejectE(res).constraints)==len(d.constraints)):
#         return []
#       else:
#         return res


#   ll = ll.removeListOfConstraints(redundants2)
#   return []





In [ ]:
from copy import deepcopy
findEPrimeCall = 0

#def findEPrimeReification(L,Y,d):
def findEPrime(L,Y,d):
  ds = len(d.constraints)

  if(ds==0 or ds==1):
    return [], None, None


  C = deepcopy(L.ConstraintsIncludedInY(Y))

  vars = { id:  cp.intvar(C.problem_data[id]["domain"][0],C.problem_data[id]["domain"][1], shape=1, name=C.problem_data[id]["name"])  for id in C.problem_data.keys() }

  model4 = cp.Model()


  bools = [cp.boolvar(name="bool"+str(i+1)) for i in range(len(d.constraints))]



  elementary = [c for c in d.constraints if isinstance(c,Constraint)]
  Conjs = [c for c in d.constraints if isinstance(c,Conjunction)]

  cpt = 0
  for c in elementary:
    c.prepareCpModelConstraintReification(vars, model4, bools[cpt])
    cpt+=1

  for c in Conjs:
    c.prepareCpModelConstraintReification(vars, model4, bools[cpt])
    cpt+=1

  #objvar = cp.intvar(1, len(bools)-1, shape=1, name="obj")

  #model += objvar >= objvar.lb
  model4 += cp.all([(sum(bools) >= 1),(sum(bools) < len(d.constraints))])



  for c in C.constraints:
    c.prepareCpModelConstraint(vars,model4)

  for v in vars.keys():
      model4 += vars[v] >= vars[v].lb


  #model += sum(bools) != 0

  if(model4.solve(solver="exact")):
      e = []
      for v in vars.keys():
        e.append(
            (v, vars[v].name, vars[v].value())
          )
      return projection(e, Y), C, d
  else:
    return [], C, d

In [ ]:
# import numpy as np

# n = 4
# ub = n * n + 1

# vars_ids =   [i+1 for i in range(n)]
# vars_names = ["m"+str(i+1) for i in range(n)]
# vars_types = ["marker" for i in range(n)]
# vars_domains = [(0,0)]
# vars_domains = vars_domains + [(1,ub) for i in range(n-1)]
# types = ["marker"]


# problem_data = {i+1: {"domain": (1,ub), "name": "m"+str(i+1),"type": "marker"} for i in range(n)}
# problem_data[1]["domain"] = (0,0)
# print(problem_data)






# def eqDist(a,b,c,d):
#   return operator.eq( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

# def neDist(a,b,c,d):
#   return operator.ne( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))


# def eqDist3(repeat,b,c):
#   return operator.eq( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

# def neDist3(repeat,b,c):
#   return operator.ne( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))


# language = [
#     (operator.ge, 2, False,None),
#     (operator.le, 2, False,None),
#     (operator.gt, 2, False,None),
#     (operator.lt, 2, False,None),
#     (operator.ne, 2, True,None),
#     (operator.eq, 2, True,None),
#     (eqDist, 4, True,None),
#     (neDist, 4, True,None),
#     (eqDist3, 3, False,None),
#     (neDist3, 3, False,None)
# ]

# B = Basis(problem_data , language)
# B.build()
# # B = B.removeDuplicates()

# targetconstraints = []


# # there is an increasing constraint between markers
# for i in range(n-1):
#   targetconstraints.append(Constraint([vars_ids[i],vars_ids[i+1]], operator.lt, 2,False))


# distances = list(combinations(range(1,n+1), 2))


# for i in range(len(distances)-1):
#   for j in range(i+1,len(distances)):
#     p = list(distances[i])+list(distances[j])
#     pp = np.array(p)
#     if(len(np.unique(pp)) == len(pp)):
#       targetconstraints.append(Constraint(p, neDist , 4,True))
#     else:
#       if(p[0]==p[2]):
#         targetconstraints.append(Constraint([p[0],p[1],p[3]], neDist3 , 3,False))
#       elif(p[0]==p[3]):
#         targetconstraints.append(Constraint([p[0],p[1],p[2]], neDist3 , 3,False))
#       elif(p[1]==p[2]):
#         targetconstraints.append(Constraint([p[1],p[0],p[3]], neDist3 , 3,False))
#       elif(p[1]==p[3]):
#         targetconstraints.append(Constraint([p[1],p[0],p[2]], neDist3 , 3,False))



# Target = Network( problem_data, targetconstraints)

In [ ]:
# findEPrime(Target, [1,2,3], B)

In [ ]:
#n1 = Network(problem_data,B.ConstraintsIncludedInY([1,2,3]))
# n2 = Network(problem_data,B.ConstraintsIncludedInY([1,2,3]).constraints)

#nn = joinNetworks(n1,n2)

# for c in nn.constraints:
#   print(c.rel, c.scope_ids)

In [ ]:
# findEPrime(Target, [1,2,3], n2)

## FindC

In [ ]:
def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False


def existAlready2(res,c):

    if(ConstraintinListOfConstraints(c, res)):
        return True
    res.append(c)
    return False

def removeDuplicates(listOfConstraints):
  res = []
  nonDup = []
  for c in listOfConstraints:
    if(existAlready2(res,c)):
      continue
    else:
      nonDup.append(c)

  # print("\nBefore remove inside function: ", len(listOfConstraints))
  # print("\nAfter remove inside function: ", len(nonDup))
  return nonDup

def FindC(e,Y,L,B, TimeLimit = None, SolutionLimit = None):
  global cpt
  global newsize
  global oldsizeexamples
  global newsizeexamples
  global nbExamplesc
  global nbExamplesa
  global Exampletimes

  delta = []

  delta = B.ConstraintsIsExactlyY(Y)

  tmpp = delta.networkOfConstraintsThatRejectE(e)

  # tmpp.constraints = removeDuplicates(tmpp.constraints)

  delta = joinNetworks( delta , tmpp)

  global findEPrimeCall
  while(True):

    print("\nSTARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    print("\n\nDelta before findEPrime: \n")
    for c in delta.constraints:
      print(c.rel, c.scope_ids)


    print("\n\nL[Y] before findEPrime: \n")
    for c in L.ConstraintsIncludedInY(Y).constraints:
      print(c.rel, c.scope_ids)


    stg = time.time()
    ep, ll, d = findEPrime(L,Y,delta)
    #register example sizes and times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    print("\n Example: ", ep)

    print("\n\nDelta returned from findEPrime: \n")
    for c in delta.constraints:
      print(c.rel, c.scope_ids)


    print("\n\nL[Y] returned from findEPrime: \n")
    for c in L.ConstraintsIncludedInY(Y).constraints:
      print(c.rel, c.scope_ids)

    print("\nENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    findEPrimeCall+=1
    if ep == []:
      print("\n A new constraint was added to the being learned network !!!:\n")
      elementary = [c for c in delta.constraints if isinstance(c,Constraint)]
      Conjs = [c for c in delta.constraints if isinstance(c,Conjunction)]

      if(len(elementary) != 0):
        c = elementary[0]
      else:
        min = 100000
        c = None
        for cc in Conjs:
          if(len(cc.elementaryConstraints)<min):
            min = len(cc.elementaryConstraints)
            c = cc
      #c = delta.constraints[0]


      L = L.addConstraint(c)
      newsize+=1
      newsizeexamples+=1
      print("\n Number of constraints so far: \n", len(L.constraints))
      B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(Y).constraints)
      print("\n FindC : Remaining constraints in the Basis: \n", len(B.constraints))
      # print(input())
      break
    else:
      if(ask(ep,Target)):
        delta = delta.removeConstraintsThatRejectE(ep)

        B = B.removeConstraintsThatRejectE(ep)

        print("\n FindC : B reduced in size, new size: ",len(B.constraints))
      else:

        S = FindScope(ep,[],Y,B)
        if set(S).issubset(set(Y)) and len(S)!=len(Y) : #and S!=[]:
          FindC(ep,S,L,B,TimeLimit, SolutionLimit)
        else:
          tmp1 = delta
          tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
          delta = joinNetworks(tmp1, tmp11)

          print("\nAfter join findcloop: ")
          print(len(delta.constraints))

In [ ]:
# def FindC(e,Y,L,B, TimeLimit = None, SolutionLimit = None):
#   global cpt

#   delta = []

#   delta = B.ConstraintsIsExactlyY(Y)

#   tmpp = delta.networkOfConstraintsThatRejectE(e)

#   #tmpp.constraints = removeDuplicates(tmpp.constraints)

#   delta = joinNetworks( delta , tmpp)

#   print("\n\n\n-------len(delta) before loop: ", len(delta.constraints))

#   while(True):

#     ep = findEPrime(L,Y,delta)

#     if ep == []:
#       print("\n A new constraint was added to the being learned network !!!:\n")
#       elementary = [c for c in delta.constraints if isinstance(c,Constraint)]
#       Conjs = [c for c in delta.constraints if isinstance(c,Conjunction)]

#       if(len(elementary) != 0):
#         c = elementary[0]
#       else:
#         min = 100000
#         c = None
#         for cc in Conjs:
#           if(len(cc.elementaryConstraints)<min):
#             min = len(cc.elementaryConstraints)
#             c = cc
#       #c = delta.constraints[0]


#       L = L.addConstraint(c)
#       print("\n Number of constraints so far: \n", len(L.constraints))
#       B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(Y).constraints)
#       print("\n FindC : Remaining constraints in the Basis: \n", len(B.constraints))
#       # print(input())
#       break
#     else:
#       if(ask(ep,Target)):
#         delta = delta.removeConstraintsThatRejectE(ep)

#         B = B.removeConstraintsThatRejectE(ep)

#         print("\n FindC : B reduced in size, new size: ",len(B.constraints))
#       else:

#         S = FindScope(ep,[],Y,B)
#         if set(S).issubset(set(Y)) and len(S)!=len(Y) : #and S!=[]:
#           FindC(ep,S,L,B,TimeLimit, SolutionLimit)
#         else:
#           tmp1 = delta
#           tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
#           delta = joinNetworks(tmp1, tmp11)

#           print("\nAfter join findcloop: ")
#           print(len(delta.constraints))

## QuAcq2

In [ ]:
# main

def QuAcq2(B, TimeLimit = None, SolutionLimit = None):
  global Target
  global problem_data
  global redundants
  global nbExamplesc
  global nbExamplesa
  global oldsizeexamples
  global newsizeexamples
  global Exampletimes

  print("In QuAcq2: len of constraints in B is: ", len(B.constraints))
  L = Network(problem_data,[])
  cpt = 0
  while(True):

    stg = time.time()
    e = generateExample(list(problem_data.keys()),L, B)
    #register example times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    if(e==[]):
      return True, L # convergence

    if(ask(e, Target)):

      B = B.removeConstraintsThatRejectE(e)
      print("\nQuacq B reduced in size, new size: ",len(B.constraints))
    else:
      FindC(e, FindScope(e , [] , list(problem_data.keys()), B ), L, B, TimeLimit, SolutionLimit)

  return L


# Experiments

## Purdey

In [ ]:
# # Regular vars case

# calls= 0

# from random import random

# vars_ids =   [   1 ,   2   ,  3    ,  4     ,   5  ,  6  ,    7   ,   8 ,    9  ,   10  ,   11   ,  12]
# vars_names = ["family1","family2","family3","family4","bought1","bought2","bought3","bought4","paid1","paid2","paid3","paid4"]
# vars_types = ["family","family","family","family","bought","bought","bought","bought","paid","paid","paid","paid"]
# vars_domains = [(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4)]
# types = ["family","bought","paid"]


# problem_data = {
#   id: {
#       "domain": vars_domains[id-1],
#        "type": vars_types[id-1],
#        "name": vars_names[id-1]
#   } for id in vars_ids
# }



# language = [ (operator.ge,2, False, None),
#              (operator.le,2, False, None),
#              (operator.lt,2, False, None),
#              (operator.gt,2, False, None),
#              (operator.ne,2,True, None),
#              (operator.eq,2,True, None),
#              ]
# constraints = [

#     # alldiff family
#     Constraint([1,2], operator.ne, 2,True),
#     Constraint([1,3], operator.ne, 2,True),
#     Constraint([1,4], operator.ne, 2,True),
#     Constraint([2,3], operator.ne, 2,True),
#     Constraint([2,4], operator.ne, 2,True),
#     Constraint([3,4], operator.ne, 2,True),

#     # alldiff bought
#     Constraint([5,6], operator.ne, 2,True),
#     Constraint([5,7], operator.ne, 2,True),
#     Constraint([5,8], operator.ne, 2,True),
#     Constraint([6,7], operator.ne, 2,True),
#     Constraint([6,8], operator.ne, 2,True),
#     Constraint([7,8], operator.ne, 2,True),

#     # alldiff paid
#     Constraint([9,10], operator.ne, 2,True),
#     Constraint([9,11], operator.ne, 2,True),
#     Constraint([9,12], operator.ne, 2,True),
#     Constraint([10,11], operator.ne, 2,True),
#     Constraint([10,12], operator.ne, 2,True),
#     Constraint([11,12], operator.ne, 2,True),

#     #############################

#     Constraint([1,5], operator.gt, 2,True),
#     Constraint([5,7], operator.gt, 2,True),
#     Constraint([5,8], operator.gt, 2,True),
#     Constraint([12,10], operator.lt, 2,True),
#     Constraint([5,11], operator.eq, 2,True),
#     Constraint([3,5], operator.eq, 2,True),
#     Constraint([8,12], operator.eq, 2,True),
#     Constraint([2,7], operator.eq, 2,True),
#     Constraint([7,10], operator.eq, 2,True),

#  ]



# B = Basis(problem_data, language)
# B.build()


# Target = Network(problem_data , constraints)

In [ ]:
# len(B.constraints), len(Target.constraints)

In [ ]:
# Target.getAllSolutions()

In [ ]:
# Qc = 0
# Qa = 0

# nbExamplesa = 0
# nbExamplesc = 0


# Exampletimes = []
# Examplesizes = []


# oldsize = 0
# newsize = 0

# oldsizeexamples = 0
# newsizeexamples = 0

# waittimes = []
# maxwaittimeold = time.time()

# res = QuAcq2(B)

In [ ]:
# len(res[1].constraints) ,[(c.rel,c.scope_ids) for c in res[1].constraints]

In [ ]:
# res[1].getAllSolutions()

In [ ]:
# print("Qc: ",Qc,"\nQa: ",Qa,"\nnbExamplesa: ",nbExamplesa,"\nnbExamplesc: ",nbExamplesc)

# print("len(Exampletimes): ",len(Exampletimes),"\nlen(Examplesizes): ",len(Examplesizes))


# print("oldsize,newsize: ",oldsize,newsize)

In [ ]:
# averageGenerationTime = sum(Exampletimes)/len(Exampletimes)
# averageExampleSize = sum(Examplesizes) / len(Examplesizes)

# cumulativeWaitingUntilLearning = sum(Exampletimes[:nbExamplesa])
# totalCumulativeWaiting = sum(Exampletimes[:nbExamplesc])


In [ ]:
# averageGenerationTime, averageExampleSize,cumulativeWaitingUntilLearning,totalCumulativeWaiting

In [ ]:
# len(waittimes), max(waittimes)

## 10 runs

In [ ]:
from random import random
  # Regular vars case
nbCallll = 0

ressss = []

vars_ids =   [   1 ,   2   ,  3    ,  4     ,   5  ,  6  ,    7   ,   8 ,    9  ,   10  ,   11   ,  12]
vars_names = ["family1","family2","family3","family4","bought1","bought2","bought3","bought4","paid1","paid2","paid3","paid4"]
vars_types = ["family","family","family","family","bought","bought","bought","bought","paid","paid","paid","paid"]
vars_domains = [(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4),(1,4)]
types = ["family","bought","paid"]


problem_data = {
  id: {
      "domain": vars_domains[id-1],
      "type": vars_types[id-1],
      "name": vars_names[id-1]
  } for id in vars_ids
}



language = [ (operator.ge,2, False, None),
            (operator.le,2, False, None),
            (operator.lt,2, False, None),
            (operator.gt,2, False, None),
            (operator.ne,2,True, None),
            (operator.eq,2,True, None),
            ]
constraints = [

    # alldiff family
    Constraint([1,2], operator.ne, 2,True),
    Constraint([1,3], operator.ne, 2,True),
    Constraint([1,4], operator.ne, 2,True),
    Constraint([2,3], operator.ne, 2,True),
    Constraint([2,4], operator.ne, 2,True),
    Constraint([3,4], operator.ne, 2,True),

    # alldiff bought
    Constraint([5,6], operator.ne, 2,True),
    Constraint([5,7], operator.ne, 2,True),
    Constraint([5,8], operator.ne, 2,True),
    Constraint([6,7], operator.ne, 2,True),
    Constraint([6,8], operator.ne, 2,True),
    Constraint([7,8], operator.ne, 2,True),

    # alldiff paid
    Constraint([9,10], operator.ne, 2,True),
    Constraint([9,11], operator.ne, 2,True),
    Constraint([9,12], operator.ne, 2,True),
    Constraint([10,11], operator.ne, 2,True),
    Constraint([10,12], operator.ne, 2,True),
    Constraint([11,12], operator.ne, 2,True),

    #############################

    Constraint([1,5], operator.gt, 2,False),
    Constraint([5,7], operator.gt, 2,False),
    Constraint([5,8], operator.gt, 2,False),
    Constraint([12,10], operator.lt, 2,False),
    Constraint([5,11], operator.eq, 2,True),
    Constraint([3,5], operator.eq, 2,True),
    Constraint([8,12], operator.eq, 2,True),
    Constraint([2,7], operator.eq, 2,True),
    Constraint([7,10], operator.eq, 2,True),

]



Borigin = Basis(problem_data, language)
Borigin.build()
Borigin = Borigin.removeDuplicates()
lbasis = len(Borigin.constraints)

Targetorigin = Network(problem_data , constraints)
Targetorigin = Targetorigin.removeDuplicates()
ltarget = len(Targetorigin.constraints)




while(nbCallll<10):
  nbCallll+=1


  B = deepcopy(Borigin)
  Target = deepcopy(Targetorigin)

  Qc = 0
  Qa = 0

  nbExamplesa = 0
  nbExamplesc = 0


  Exampletimes = []
  Examplesizes = []


  oldsize = 0
  newsize = 0

  oldsizeexamples = 0
  newsizeexamples = 0

  waittimes = []


  ttts = time.time()
  maxwaittimeold = time.time()

  res = QuAcq2(B)
  ttte = time.time() - ttts

  averageGenerationTime = sum(Exampletimes)/len(Exampletimes)
  averageExampleSize = sum(Examplesizes) / len(Examplesizes)

  cumulativeWaitingUntilLearning = sum(Exampletimes[:nbExamplesa])
  totalCumulativeWaiting = sum(Exampletimes[:nbExamplesc])



  ressss.append({
        "Asolution":res[1].solve(),
        "isAccepted":Target.isAccepted(res[1].solve()),
        "isTargetEquivalentToLearned": Target.isEquivalentTo(res[1]),
        "Learned": res[1],
        "sizeOfL":len(res[1].constraints),
        "sizeOfBasis":lbasis,
        "sizeOfTarget":ltarget,
        "QA":Qa,
        "QC":Qc,
        "nbExamplesa":nbExamplesa,
        "nbExamplesc":nbExamplesc,
        "tmax":max(waittimes),
        "t":averageGenerationTime,
        "averageExampleSize":averageExampleSize,
        "timeA":cumulativeWaitingUntilLearning,
        "timeC":totalCumulativeWaiting,
        "runningTime":ttte
    })

  print("\n\n--------------Another entry was added to ressss--------------\n\n")



In QuAcq2: len of constraints in B is:  396

 a COMPLETE query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  387

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  381

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  0 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[3, 4], [3, 4]]
[<built-in function le>, <built-in function lt>] [[4, 3], [4, 3]]
<built-in function ne> [3, 4]


L[Y] before findEPrime: 


 Example:  [(3, 'family3', 1), (4, 'family4', 3)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[3, 4], [3, 4]]
[<built-in function le>, <built-in function lt>] [[4, 3], [4, 3]]
<built-in function ne> [3, 4]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND 

<ipython-input-15-4d80adbd618e>:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")



STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  20 ---------------------





Delta before findEPrime: 

<built-in function le> [8, 5]
[<built-in function le>, <built-in function lt>] [[8, 5], [8, 5]]


L[Y] before findEPrime: 


 Example:  [(8, 'bought4', 3), (5, 'bought1', 3)]


Delta returned from findEPrime: 

<built-in function le> [8, 5]
[<built-in function le>, <built-in function lt>] [[8, 5], [8, 5]]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall:  20 ---------------------




 a PARTIAL query was sent, the answer was NO

After join findcloop: 
1

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  21 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[8, 5], [8, 5]]


L[Y] before findEPrime: 


 Example:  []


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[8, 5], [8, 5]]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND fin

<ipython-input-15-4d80adbd618e>:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")



STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  23 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[9, 10], [9, 10]]
[<built-in function le>, <built-in function lt>] [[10, 9], [10, 9]]
<built-in function ne> [9, 10]


L[Y] before findEPrime: 


 Example:  [(9, 'paid1', 1), (10, 'paid2', 3)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[9, 10], [9, 10]]
[<built-in function le>, <built-in function lt>] [[10, 9], [10, 9]]
<built-in function ne> [9, 10]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall:  23 ---------------------




 a PARTIAL query was sent, the answer was YES

 FindC : B reduced in size, new size:  196

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  24 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[9, 10], [9, 10]]
<built-in function ne> [9, 10]


L[Y] before findEPrime: 


...


STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  193 ---------------------





Delta before findEPrime: 

<built-in function le> [8, 12]
[<built-in function le>, <built-in function lt>] [[8, 12], [8, 12]]
[<built-in function le>, <built-in function eq>] [[8, 12], [8, 12]]


L[Y] before findEPrime: 


 Example:  [(8, 'bought4', 2), (12, 'paid4', 3)]


Delta returned from findEPrime: 

<built-in function le> [8, 12]
[<built-in function le>, <built-in function lt>] [[8, 12], [8, 12]]
[<built-in function le>, <built-in function eq>] [[8, 12], [8, 12]]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall:  193 ---------------------




 a PARTIAL query was sent, the answer was NO

After join findcloop: 
1

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  194 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function eq>] [[8, 12], [8, 12]]


L[Y] before findEPrime: 


 Example:  []


Delta returned from findEPr

<ipython-input-15-4d80adbd618e>:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")



STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  221 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[6, 7], [6, 7]]
[<built-in function le>, <built-in function lt>] [[7, 6], [7, 6]]
<built-in function ne> [6, 7]


L[Y] before findEPrime: 


 Example:  [(6, 'bought2', 1), (7, 'bought3', 3)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[6, 7], [6, 7]]
[<built-in function le>, <built-in function lt>] [[7, 6], [7, 6]]
<built-in function ne> [6, 7]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall:  221 ---------------------




 a PARTIAL query was sent, the answer was YES

 FindC : B reduced in size, new size:  293

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  222 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[6, 7], [6, 7]]
<built-in function ne> [6, 7]


L[Y] before findEPrime: 


 Exam

<ipython-input-15-4d80adbd618e>:176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")


Streaming output truncated to the last 5000 lines.
before remove reudndants: 

82
after remove reudndants: 

20


--------------Another entry was added to ressss--------------


In QuAcq2: len of constraints in B is:  396

 a COMPLETE query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  387

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  381

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  331 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[3, 4], [3, 4]]
[<built-in function le>, <built-in function lt>] [[4, 3], [4, 3]]
<built-in function ne> [3, 4]


L[Y] before findEPrime: 


 Example:  [(3, 'family3', 1), (4, 'family4', 3)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[3

In [ ]:
import pickle

with open("respurdey10runs.txt", "wb") as f:
  pickle.dump(ressss,f)


In [ ]:
ressss

[{'Asolution': [(1, 'family1', 4),
   (2, 'family2', 2),
   (3, 'family3', 3),
   (4, 'family4', 1),
   (5, 'bought1', 3),
   (6, 'bought2', 4),
   (7, 'bought3', 2),
   (8, 'bought4', 1),
   (9, 'paid1', 4),
   (10, 'paid2', 2),
   (11, 'paid3', 3),
   (12, 'paid4', 1)],
  'isAccepted': True,
  'sizeOfL': 20,
  'sizeOfBasis': 396,
  'sizeOfTarget': 27,
  'QA': 129,
  'QC': 130,
  'nbExamplesa': 62,
  'nbExamplesc': 64,
  'tmax': 1.1353514194488525,
  't': 0.11853310465812683,
  'averageExampleSize': 6.730769230769231,
  'timeA': 5.701156139373779,
  'timeC': 7.586118698120117,
  'runningTime': 14.264078855514526},
 {'Asolution': [(1, 'family1', 4),
   (2, 'family2', 2),
   (3, 'family3', 3),
   (4, 'family4', 1),
   (5, 'bought1', 3),
   (6, 'bought2', 4),
   (7, 'bought3', 2),
   (8, 'bought4', 1),
   (9, 'paid1', 4),
   (10, 'paid2', 2),
   (11, 'paid3', 3),
   (12, 'paid4', 1)],
  'isAccepted': True,
  'sizeOfL': 21,
  'sizeOfBasis': 396,
  'sizeOfTarget': 27,
  'QA': 128,
  'QC': 

In [ ]:
[r["isAccepted"] for r in ressss]

[True, True, True, True, True, True, True, True, True, True]